In [1]:
from CurriculumVitae import *
from IPython.display import display, HTML, Markdown, clear_output
from datetime import date
import json
from jobs_skills_weights import get_jobs, get_raw_job_details
import skill_weights, synonyms, skill_cat
import pandas as pd # TODO: Not needed for final export

In [2]:
my_resume = Resume(
    person=Person(name="Zach Allen", pronouns="He/Him", 
                    contact_info=ContactInfo(
                       email="fractalmachinist@gmail.com", 
                       phone="+1 (509)438-8146", 
                       link="https://fractalmachini.st",
                       link2="https://linkedin.com/in/zachallen-fractalmachinist/")),
    #                                                                                                                                                                  # total / unique
    headline=' '.join(["Machine Learning Researcher & Data Engineer with 5 years experience in collaborative AI innovation & infrastructure implementation at scale.", # 27.4% / 13.0%
                       "Deep Learning frameworks: expert in Python & TensorFlow since 2015/2017.",                                                                     # 6.20% / 6.20%
                       "Analyzed, tested, and optimized innovative Machine Learning algorithms for unbalanced datasets (>1000:1), raising rare-case recognition",      # 10.1% / 3.67%
                       "from 15% to 90% with no loss in accuracy.",                                                                                                    #     ^cont
                       "Passionate about collaborating & working closely with multidisciplinary partners to shape the future towards global sustainability.",          # 8.01% / 4.89%
                       "Independently implemented and deployed terabyte-scale AWS genomics pipeline infrastructure in 4 months part-time,",                            # 7.52% / 2.64%
                       "from no prior AWS or genomics experience.",                                                                                                    #     ^cont
                       "Certified in data-driven integrations of business needs, software engineering, project management, and software development best practices.",  # 4.88% / 4.88%
                       "Excellent writing, speaking, presenting, and technical communication skills.",                                                                 # 3.58% / 3.58%
                       ]))

In [3]:
my_resume.education = [Occupation(
    title="Bachelor's in Computer Science", location="Utah, USA (Online)",
    timespan=Between(start=date(2019, 2, 1), end=date(2022, 8, 31)),
    subtitle="Western Governors University", 
    headline="Focused on Machine Learning and Project Management in a fully remote environment",
    skills=my_resume.Skills("Remote Work", "Project Management", "Machine Learning", "Communication"),
    sub_tasks=[
        Effort(title="Diamond Price Prediction Model",
               headline="Proposed, implemented, and documented a diamond price prediction ML model.",
               website="https://github.com/FractalMachinist/WGU-C968",
               skills=my_resume.Skills("Machine Learning", "Git", "Software Engineering", "Jupyter", "Analysis", "Statistics", "Feature Engineering", "Communication","Seaborn", "Data Visualization"),
               achievements=[
                  Achievement(
                       headline="Demonstrated integration of project management and software engineering best practices.",
                       skills=my_resume.Skills("Project Management", "Communication", "Documentation", "Managing Requirements", "Data Driven", "End to End", "Technical Communication")),
                   Achievement(
                       headline="Developed simple Data Pipeline in python/pandas.",
                       skills=my_resume.Skills("Python", "Data", "Software Engineering", "NumPy", "SciPy", "Pandas")),]

        ), 
        Effort(title="Advanced Java Concepts",
               headline="Developed appointment scheduling and customer database tool in Java.",
               website="https://github.com/FractalMachinist/C195-Scheduling-App",
               skills=my_resume.Skills("Java", "Software Engineering", "Git", "SQL"),
               achievements=[
               Achievement(
                   headline="Encapsulated Observable state and database connection state into a shared wrapper class, streamlining inheritance, error handling, and UI/Database auto-updates.",
                   skills=my_resume.Skills("Java", "JavaFX", "JDBC", "Integration", "API Design", "MySQL")),
               Achievement(
                   headline="Paid technical debt by finalizing project documentation.",
                   skills=my_resume.Skills("Documentation", "Communication", "Ownership", "SDLC", "Project Management", "Scope"))
               ]),    
    ],
    
    achievements=[
        
        Certification(name="CompTIA Project+",
                      headline="Demonstrated understanding of Project Management roles, processes, and documentation.",
                      issuer="Pearson VUE", issue_date=date(2019, 7, 24),
                      website="https://wsr.pearsonvue.com/testtaker/authenticate/AuthenticateScoreReport.htm",
                      confirmation_info={"Registration":"358639011", "Validation":"155946649"},
                      skills=my_resume.Skills("Project Management", "Communication", "Managing Requirements", "SDLC", "Integration", "Agile", "Scope", "Stakeholder Support")),
        
        Certification(name="IT Information Library Foundations Certification (ITIL)", 
                      headline="Demonstrated understanding of designing, deploying, maintaining, and retiring IT resources.",
                      issuer="AXELOS", issue_date=date(2020, 8, 1),
                      skills=my_resume.Skills("Project Management", "Managing Requirements", "SDLC", "Integration")),
        
        Certification(name="Site Development Associate",
                      headline="Demonstrated ability to design and build websites.",
                      issuer="CIW", issue_date=date(2019, 2, 1),
                      skills=my_resume.Skills("Web Development", "HTML", "CSS", "JavaScript",)),
        
        Achievement(headline="Excellence Award for Communication Applications",
                    portfolio_link="https://github.com/FractalMachinist/Rust_Business_Presentation",
                    skills=my_resume.Skills("Communication", "Documentation", "Technical Communication")),
        
    ])]

In [4]:
my_resume.employment = [
    Occupation(title="Sofware Engineer", location="St. Louis, Missouri, USA", website="https://PlutonBio.com",
               timespan=Between(start=date(2021, 3, 1), end=date(2021, 8, 31)),
               subtitle="Pluton Biosciences", supervisor=Person(name="Dr. Boahemaa Adu-Oppong", pronouns="She/Her",
                                                                contact_info=ContactInfo(email="BAdu-Oppong@plutonbio.com")),
               headline="Data Engineering supporting Bioinformatics Research",
               skills=my_resume.Skills("Remote Work", "Big Data", "Bioinformatics", "Genetics", "Multidisciplinary", "Motivated"),
               sub_tasks=[
                   Effort(
                       title="AWS Genomics Pipeline",
                       headline="Independently designed, implemented, and deployed Terabyte-scale AWS genomics pipeline in 4 months part time, from no prior genomics or AWS experience.",
                       skills=my_resume.Skills("AWS", "Cloud Computing", "Data", "Big Data", "Large Scale", "Tooling", "Infrastructure", "End to End", "Motivated", "Multidisciplinary", "Problem Solving", "Remote Work", "Innovation", "Curious", "Ownership", "Project Management", "SDLC", "Data Architecture", "Data Infrastructure"),
                       achievements=[
                           Achievement(
                               # title="Pipeline Design",
                               headline="Designed, implemented, and documented gRPC+ProtoBuf pipeline architecture.",
                               skills=my_resume.Skills("Python", "Bash", "Infrastructure", "Data", "Big Data", "Large Scale", "Tooling", "Integration", "Automation", "Prototyping", "Workflows", "Ownership", "Innovation", "Problem Solving"),
                           ),
                           Achievement(
                               # title="Streamlined Pipeline Development Lifecycle",
                               headline="Developed, documented, and utilized frameworks for automating Computational Genomics tool containerization.",
                               skills=my_resume.Skills("Python", "Bash", "Software Engineering", "Frameworks", "Infrastructure", "Tooling", "SDLC", "Automation", "Prototyping", "Workflows", "Documentation", "Innovation", "Problem Solving")
                           ),
                           Achievement(
                               # title="User-Friendly Query Library",
                               headline="Designed, implemented, developed training for, and documented an in-house Python+R library and interface enabling (non-CS) biologists to easily deploy multi-stage genomics queries without relying on the CS team.",
                               skills=my_resume.Skills("Python", "R", "API Design", "Libraries", "Tooling", "Automation", "Workflows", "Documentation", "Problem Solving", "Value Creation", "Technical Communication")
                           ),
                           Achievement(
                               # title="Aligned Deliverables with Stakeholder Needs",
                               headline="Collaborated with SMEs to gather, interpret, and meet data pipeline requirements. Interfaced with Bioinformatics expert for genomics tool selection.",
                               skills=my_resume.Skills("Managing Requirements", "Communication", "Project Management", "Collaboration", "Multidisciplinary", "Scope", "Stakeholder Support", "Technical Communication")
                           ),
                       ],
                   ),
                       
                   Effort(
                       title="Containerized Deployment & Autoscaling",
                       headline="Researched, containerized, documented, and deployed 10+ Computational Genomics tools to AWS ECS and EC2.",
                       skills=my_resume.Skills("Integration", "Bash", "Linux", "AWS", "Containerization", "Large Scale", "Frameworks", "Infrastructure", "Bioinformatics", "Genetics", "ECS","EC2"),
                       achievements=[
                           Achievement(
                               headline="Constructed and tuned EC2&ECS autoscaling, maximizing cost efficiency and cluster responsiveness.",
                               skills=my_resume.Skills("AWS", "Infrastructure", "Budget", "Monitoring", "Tracking", "Reporting")
                           ),
                           Achievement(
                               headline="Detected, diagnosed, and resolved compute resource inefficiencies.",
                               skills=my_resume.Skills("Optimization", "Software Engineering", "Computer Science", "Debugging", "Computer Engineering", "Iteration", "Monitoring", "Tracking", "Reporting")
                           ),
                           Achievement(
                               # title="Spearheaded SDLC & CI/CD",
                               headline="Led the CS team in adopting SDLC tools like Git, AWS CodeCommit, and Docker / AWS Elastic Container Registry. Championed utilization of Microsoft Teams Kanban tools for project&process management.",
                               skills=my_resume.Skills("Leadership", "SDLC", "Git", "Containerization", "Testing", "Integration", "Frameworks", "Project Management", "Teamwork", "Workflows", "CI/CD", "Organizational Skills")
                           ),
                       ]
                   ),      
                   Achievement(
                       # title="Microbial Taxonomy Visualizations in R",
                       headline="Worked closely with Microbiologists & Geneticists to perform analysis and visualization of microbial taxonomy data in R.",
                       skills=my_resume.Skills("R", "Analysis", "Statistics", "Communication", "Collaboration", "Multidisciplinary", "Teamwork", "Iteration", "Data Visualization", "Technical Communication")
                   ),
               ],
    ),
    
    Occupation(title="Embedded Systems Engineer", location="Allentown, Pennsylvania, USA", website="https://AppliedSeparations.com",
               timespan=Between(start=date(2019, 1, 1), end=date(2020, 1, 25)),
               subtitle="Applied Separations", supervisor=Person(name="Aaron Allen", pronouns="He/Him", 
                                                                contact_info=ContactInfo(email="mnrnln@gmail.com")),
               
               headline="Developed custom C++ / Arduino pump control software",
               achievements=[
                   Achievement(headline="Designed and developed pump control software for chromatography and analytical chemistry systems in C and C++.",
                               skills=my_resume.Skills("C", "C++", "Software Engineering", "SDLC", "Microcontrollers", "Multidisciplinary")),
                   
                   Achievement(headline="Wrote a simple heuristic scheduler & virtual threading to manage real-time (60Hz+) pump control, touch screen input, and data/control communication, all on a single Arduino Mega.",
                               skills=my_resume.Skills("Algorithms", "Optimization", "Software Engineering", "Statistics", "Computer Science", "Microcontrollers", "Computer Engineering")),
                   
                   Achievement(headline="Designed and validated mass delivery tracking, prediction, and smoothing algorithms, to handle nonlinear feedback delay.",
                               skills=my_resume.Skills("Algorithms", "Algebra", "Software Engineering", "Testing", "Computer Science", "Prototyping", "Analysis", "Monitoring", "Reporting")),
                   
                   Achievement(headline="Interfaced with separate development team in charge of chromatography control software. Collected and implemented requirements from instrumentation engineer.",
                               skills=my_resume.Skills("Communication", "Managing Requirements", "Project Management", "SDLC", "Documentation", "Collaboration", "Ownership", "Integration", "Leadership", "Git", "Reporting", "Technical Communication"))]),
              
    Occupation(title="Service Writer", location="Richland, Washington, USA", website="https://AlphaComputerCenter.com",
               timespan=Between(start=date(2017, 12, 1), end=date(2018, 11, 30)),
               subtitle="Alpha Computer Center", supervisor=Person(name="Frank Ward Jr.", pronouns="He/Him",
                                                                   contact_info=ContactInfo(email="frankjr@alphacomputercenter.com",
                                                                   phone="+1 (509)946-4230")),
               
               headline="Customer service, sales, and technician support",
               achievements=[
                   Achievement(headline="Ensured customers were able to accurately understand and communicate with repair technicians, improving customer service and reducing diagnostic time.",
                               skills=my_resume.Skills("Communication")),
                   
                   Achievement(headline="Leveraged extensive Linux experience to rapidly identify and repair issues that couldn't be fixed by Mac diagnostic tools.",
                               skills=my_resume.Skills("Linux", "Bash")),
                   
                   Achievement(headline="Reduced call frequency with an informative website. See it on <a href='https://web.archive.org/web/20180113192132/http://www.alphacomputercenter.com/wordpress1/'>web archive</a>.",
                               skills=my_resume.Skills("Web Development", "HTML", "CSS", "JavaScript", "Graphic Design"))]),
              
    Occupation(title="Machine Learning Researcher (Intern)", location="Richland, Washington, USA", website="https://pnnl.gov",
               timespan=Between(start=date(2017, 1, 1), end=date(2017, 5, 31)),
               subtitle="Pacific Northwest National Laboratory", supervisor=Person(name="Dr. Enoch Yeung", pronouns="He/Him",
                                                                                   contact_info=ContactInfo(email="eyeung@ucsb.edu")),
               
               headline="ML Research and Data Engineering intern",
               achievements=[
                   Achievement(headline="Designed and tested novel Neural Network algorithms, architectures, and error formulations for NLP, image classification, and time-series data classification.",
                               skills=my_resume.Skills("Python", "TensorFlow", "Machine Learning", "Deep Learning", "Neural Network Architectures", "Feature Engineering",
                                                       "Algorithms", "Algebra", "NLP", "Data", "Curious", "Research", "Frameworks", "Computer Science",
                                                       "MatLab", "NumPy", "Software Engineering")),
                   
                   Achievement(headline="Demonstrated increased test accuracy (15% ⇾ 90% detection with higher Bayesian Confidence) on unbalanced (>1000:1) datasets, without duplication, augmentation, or batch filtering.",
                               skills=my_resume.Skills("Machine Learning", "Algebra", "Statistics", "Testing", "Research", "Algorithms", "Computer Science", "Analysis")),
                   
                   Achievement(headline="Worked independently, balancing multiple projects and deliverables with minimal mentor supervision, often meeting every two weeks.",
                               skills=my_resume.Skills("Project Management", "Communication", "Collaboration", "Reporting"))])
]

In [5]:
my_resume.projects = [
    Effort(title="Neural Cellular Segmentation",
           headline="Exploring neural cellular automata and attention (NCA+A) for medical image segmentation",
           website="https://github.com/FractalMachinist/NeuralCellularAutomataAttn",
           achievements=[
               Achievement(headline="Developed, tested, and iterated NCA+A models, balancing system resources and model size.",
                           skills=my_resume.Skills("Python", "TensorFlow", "Machine Learning", "Neural Network Architectures", "Research", "Algebra", 
                                                   "Optimization", "Statistics", "NumPy", "Analysis")),
               Achievement(headline="Created multiple tf.Data pipelines with preprocessing and data augmentation steps.",
                           skills=my_resume.Skills("Python", "Data", "Software Engineering", "TensorFlow", "Git", "Frameworks", "Computer Science")),
           ]),
    Effort(title="Interplan",
           headline="Task dependency management from a Graph Database",
           website="https://github.com/FractalMachinist/Interplan",
           achievements=[
               Achievement(headline="Developed a Neo4J+JS dependency resolution and task status tracking API.",
                           skills=my_resume.Skills("JavaScript", "Testing", "Project Management", "Data Driven", "API Design",
                                                   "Software Engineering", "Neo4J", "Apache Tinkerpop")),
               Achievement(headline="Designed intuitive React app for displaying and interacting with tasks.",
                           skills=my_resume.Skills("Web Development", "HTML", "CSS", "JavaScript", "React")),
               Achievement(headline="Packaged Node+Neo4J in Docker & Kubernetes for easy migration.",
                           skills=my_resume.Skills("Containerization", "Infrastructure"))
           ]),
    
    Effort(title="MarkNotes",
           headline="Intuitive journaling tool designed to encourage long-term review and introspection",
           website="https://github.com/FractalMachinist/MarkNotes",
           achievements=[
               Achievement(headline="Implemented MongoDB and Node API for destructuring, storing, and querying Markdown entries as semi-structured data.",
                           skills=my_resume.Skills("MongoDB", "Data", "Data Driven", "NoSQL", "API Design", "Software Engineering")),
               Achievement(headline="Designed simple, intuitive React web app interface.",
                            skills=my_resume.Skills("Web Development", "HTML", "CSS", "JavaScript", "Graphic Design")),
               Achievement(headline="Packaged Node+MongoDB in Docker & Kubernetes.",
                           skills=my_resume.Skills("Containerization", "Infrastructure"))
           ]),
    
    Effort(title="NetTimeLog",
           headline="Minimalist, accurate time tracking",
           # website="https://fractalmachini.st/demos/nettimelog",
           achievements=[
               Achievement(headline="Created time-tracking web app which records what you just completed, so you never estimate what you will do or how long it will take.",
                           skills=my_resume.Skills("Web Development", "HTML", "CSS", "JavaScript"))])
]

# Validate Resume Configuration

In [6]:
resume_skills = pd.DataFrame([{
    "skill":skill.name.lower(),
    "instances":skill.get_num_instances()
} for skill in my_resume.skills]).set_index("skill")

In [7]:
synonyms._refresh()
skill_cat._refresh()

resume_skills_comparisons = resume_skills.copy()
resume_skills_comparisons["non root"] = resume_skills_comparisons.isin(synonyms.non_root_synonyms)
resume_skills_comparisons["non category"] = ~resume_skills_comparisons.index.isin(skill_cat.skill_to_categories)
display(resume_skills_comparisons.query("`non root` or `non category`"))
[skill for skill in skill_cat.skill_to_categories.keys() if skill not in resume_skills_comparisons.index]

,instances,non root,non category
skill,,,


['usability',
 'oop',
 'azure databricks',
 'spark',
 'plotly',
 'jenkins',
 'microsoft azure',
 'strategy',
 'product design',
 'product management',
 'cancer']

# Conditional Resume Construction

In [8]:
jobs = get_jobs(bookmarked=False)
raw_job_details = get_raw_job_details(jobs)
job_descriptions = skill_weights.get_job_descriptions(raw_job_details)

In [9]:
skill_cat._refresh()
synonyms._refresh()
job_skill_counts = skill_weights.get_job_skill_counts(raw_job_details)
job_skill_weights = skill_weights.get_job_skill_weights(raw_job_details, collapse_categories=False)

## Investigation & Review

In [10]:
job_skill_shares_of_total = (job_skill_counts[["count"]] / job_skill_counts[["count"]].sum()).rename(columns={"count":"share of total"})
skill_shares = job_skill_shares_of_total.groupby(level="skill").sum()

In [11]:
skill_shares.loc["optimization", :]

share of total    0.006622
Name: optimization, dtype: float64

### Review against Job Descriptions

In [12]:
def compare_listing_vs_generated_resume(job_id:str):
    # Step 0: Collect information
    selected_job_description = job_descriptions.loc[job_id, "job description"].replace('\n', '<br/>').replace('•', '- ')
    selected_skill_text_counts = job_skill_counts.loc[job_id, :].groupby(level=["skill", "skill text"]).sum()
    selected_skill_weights = job_skill_weights.loc[job_id, :].groupby(level=["category", "skill", "skill text"]).sum()
    
    
    selected_skill_info = selected_skill_weights.join(
        selected_skill_text_counts, 
        how='outer'
    ).reorder_levels(
        ["category","skill","skill text"]
    ).sort_index()
    
    selected_skill_info["count"] = selected_skill_info["count"] / selected_skill_info.groupby("skill text").size()
    
    selected_skill_info["share of job"] = selected_skill_info["count"] / selected_skill_info["count"].sum()

    
    # Step 1: Export information to HTML w/ styling
    
    format_percent = lambda f: "{:.2%}".format(f) if f>0 else "0.--%"
    
    formatter = {"skill weight": format_percent,
                 "share of job": format_percent}
    
    with pd.option_context('display.max_rows', None):
        selected_skill_html = selected_skill_info\
                                                 .sort_values(by=[
                                                    "category",
                                                    "skill weight",
                                                    "count"
                                                 ], ascending=[True, False, False])\
                                                 .style\
                                                 .format_index(
                                                    level="skill text",
                                                    formatter=lambda st: Skill(
                                                        synonym_base="UNK",
                                                        name=st,
                                                        num_instances=0,
                                                        share=0
                                                        ).__repr_html__(
                                                            should_render_all=True
                                                        )
                                                 )\
                                                 .format(
                                                     precision=1,
                                                     na_rep="-na-",
                                                     formatter=formatter
                                                 )\
                                                 .background_gradient()\
                                                 .to_html()
        
        selected_skill_sort_html = selected_skill_info.groupby(level="skill text", dropna=False)\
                                                 .sum(min_count=1).sort_values(["skill weight", "count"], ascending=False)\
                                                 .style\
                                                 .format_index(
                                                    level="skill text",
                                                    formatter=lambda st: Skill(
                                                        synonym_base="UNK",
                                                        name=st,
                                                        num_instances=0,
                                                        share=0
                                                    ).__repr_html__(
                                                        should_render_all=True
                                                    )
                                                 )\
                                                 .format(
                                                     precision=1,
                                                     na_rep="-na-",
                                                     formatter=formatter
                                                 )\
                                                 .background_gradient()\
                                                 .to_html()
        
        selected_cat_sort_html = selected_skill_info.groupby(level="category", dropna=False)\
                                                 .sum(min_count=1).sort_values(["skill weight","count"], ascending=False)\
                                                 .style\
                                                 .format(
                                                     precision=1,
                                                     na_rep="-na-",
                                                     formatter=formatter
                                                 )\
                                                 .background_gradient()\
                                                 .to_html()
    
    constructed_resume = my_resume.__repr_html__(
        skill_text_shares=job_skill_weights.loc[job_id, "skill weight"]\
                                           .groupby(level=["skill", "skill text"]).sum(), 
        should_render_all=True
        # alt_template_prefixes={"*":"pdf"}
    )
    
    # Step 2: Construct HTML we like
    
    constructed_comparison=f"""
    <head>
        <link rel='stylesheet' href='/utilities/comparison.css'/>
    </head>
    
    <div style="display:flex; height:100%;">
        <div class="content">{selected_job_description}</div>
        <div class="content shrink">{selected_skill_html}</div>
        <div class="content shrink" style="display:flex; flex-direction:column; height:100%;">
            <div class="skill_sort">{selected_skill_sort_html}</div>
            <div class="cat_sort">{selected_cat_sort_html}</div>
        </div>
        <div class="content grow">{constructed_resume}</div>
    </div>
    """
    
    with open("docs/compare.html", 'w+') as compare_f:
        compare_f.write(constructed_comparison)

In [13]:
job_id=jobs.index[4]
compare_listing_vs_generated_resume(job_id)

In [21]:
for job_id, skill_text_shares in job_skill_weights.groupby(level=["id", "skill", "skill text"]).sum().groupby(level="id"):
    skill_text_shares = skill_text_shares.droplevel("id")
    
    print(job_id)
    
    display(HTML(job_descriptions.loc[job_id, "job description"]))
    display(skill_text_shares.query("`skill weight` > 0").sort_values(by="skill weight", ascending=False).head(60))
    display(skill_text_shares.query("`skill weight` == 0").sort_values(by="skill weight", ascending=False).head(60))
    
    my_resume.write_html_to_file(
        filepath="docs/test.html",
        skill_text_shares=skill_text_shares["skill weight"],
        alt_template_prefixes={"*":"pdf"}
    )
    
    
    break

0124e218-2d33-41e8-907b-5228ea386455


,,skill weight
skill,skill text,
machine learning,Artificial Intelligence,0.066951
data,Data,0.051758
seaborn,Seaborn,0.041229
python,Python,0.038713
numpy,NumPy,0.037707
monitoring,Monitoring,0.037561
large scale,Large Scale,0.037077
communication,Communication Skills,0.036333
data visualization,Data Visualization,0.035716


,,skill weight
skill,skill text,
bioinformatics,Bioinformatics,0.0
budget,Budget,0.0
cancer,Cancer,0.0
ec2,EC2,0.0
ecs,ECS,0.0
genetics,Genetics,0.0
graphic design,Graphic Design,0.0
jupyter,Jupyter,0.0
matlab,Matlab,0.0


### Investigating Resume Efficiency

This section seems to be revealing significant limitations in the data I'm pulling from teal. Skills like 'C++' and 'Bash' appear to not be present in any job listing.  
  
To handle this, I'm creating a `set` of skills that I'm excluding from analysis.

In [15]:
manually_excluded_skills = {'c++','bash','javafx','jdbc','genetics','apache tinkerpop'}

In [16]:
resume_skill_density = (skill_shares["share of total"] / resume_skills["instances"]).loc[resume_skills.index].sort_values(na_position='first')
resume_skill_density = resume_skill_density.loc[~resume_skill_density.index.isin(manually_excluded_skills)]

In [17]:
resume_skill_density.head()

skill
graphic design            NaN
technical communication   NaN
react                     NaN
ecs                       NaN
ci/cd                     NaN
dtype: float64

In [18]:
# skill_shares.loc["iteration", :]

### Investingating Missing Skills

In [19]:
# Skills which are **not** in my resume (*forehead*)
missing_job_skill_shares_of_total = job_skill_shares_of_total[~job_skill_shares_of_total.index.get_level_values(2).isin(resume_skills.index)]

print("missing skills                         share of total:", f"""{missing_job_skill_shares_of_total.sum().values[0]*100:.1f}%""")
print("missing skills (non-emphasis)          share of total:", f"""{missing_job_skill_shares_of_total.query("`teal category` != 'emphasis'").sum().values[0]*100:.1f}%""")
print("missing skills                @ > 0.1% share of total:", f"""{missing_job_skill_shares_of_total.groupby(level="skill").sum().query("`share of total` > 0.001").sum().values[0]*100:.1f}%""")
print("missing skills (non-emphasis) @ > 0.1% share of total:", f"""{missing_job_skill_shares_of_total.query("`teal category` != 'emphasis'").groupby(level="skill").sum().query("`share of total` > 0.001").sum().values[0]*100:.1f}%""")

missing skills                         share of total: 52.5%
missing skills (non-emphasis)          share of total: 35.1%
missing skills                @ > 0.1% share of total: 25.3%
missing skills (non-emphasis) @ > 0.1% share of total: 9.0%


In [20]:
missing_job_skill_shares_of_total.query("`teal category` != 'emphasis'").groupby(level="skill").sum().sort_values("share of total", ascending=False).head(30)

,share of total
skill,
white space,0.006267
global,0.004966
cutting edge,0.004020
partner,0.003547
computer vision,0.003547
paid,0.003311
programs,0.002956
spark,0.002720
best practices,0.002483
